<p><b>Remember the saga of SPID 54? </b>In case you’ve forgotten its sad tale, let’s revisit:</p>
  
  ![F1_02_Activity_Monitor](img/F1_02_Activity_Monitor.jpg)

<p>There it is. 54. <b>Doing something, as usual. And of course, that something is not exactly apparent</b> from the above sp_who2 output. </p>
  
<p>This is such a major defect that when I wrote Who is Active I decided to show query text by default even if it slightly slowed things down in some cases. This can be disabled—we’ll get to that in a later post—but I don’t think anyone does that very often.</p>
  
<p>To understand the differences between the options, consider the following batch:</p>
  
<blockquote>   
<p><font face="Courier New">SELECT *        <br>FROM sys.tables</font></p>
    
<p><font face="Courier New">WAITFOR DELAY '10:10:10'</font></p>
    
<p><font face="Courier New">SELECT *        <br>FROM sys.databases        <br>GO</font></p>
 </blockquote>
  
<p>Running this batch on session 53, then running sp_WhoIsActive on another session, the following is output:</p>
  
  ![F10_01_default](img/F10_01_default.jpg)

<p>The [sql_text] column, by default, outputs the text of the <i>statement</i> that is currently running. We can instead decide to see the entire <i>batch</i>, by setting @get_full_inner_text:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @get_full_inner_text = 1</font></p>
 </blockquote>
  
  ![F10_01_full_text](img/F10_01_full_text.jpg)

<p><b>Why is this option called “get full inner text” rather than “get full batch?” </b>It’s assumed that in most systems (hopefully) the majority of code running will be stored procedure calls rather than ad hoc batches. Let’s see what happens in that case:</p>
  
<blockquote>   
<p><font face="Courier New">USE tempdb       <br>GO</font></p>
    
<p><font face="Courier New">CREATE PROC test_proc       <br>AS        <br>BEGIN        <br>&nbsp;&nbsp;&nbsp; SELECT *         <br>&nbsp;&nbsp;&nbsp; FROM sys.tables</font></p>
    
<p><font face="Courier New">&nbsp;&nbsp;&nbsp; WAITFOR DELAY '10:10:10'</font></p>
    
<p><font face="Courier New">&nbsp;&nbsp;&nbsp; SELECT *        <br>&nbsp;&nbsp;&nbsp; FROM sys.databases        <br>END        <br>GO</font></p>
    
<p><font face="Courier New">EXEC test_proc       <br>GO        <br></font></p>
 </blockquote>
  
<p>Running Who is Active with the default mode outputs pretty much the same exact thing as before:</p>
  
  ![F10_03_default](img/F10_03_default.jpg)

<p>Setting the @get_full_inner_text option now returns the entire text of the stored procedure—or the “full inner text.”</p>
  
  ![F10_03_full_text](img/F10_03_full_text.jpg)

<p><b>If you’re wondering what &lt;?query means and why the results are returned in XML format</b>, consider a nicely laid-out piece of code like the following:</p>
  
<blockquote>   
<p><font face="Courier New">SELECT        <br>&nbsp;&nbsp;&nbsp; *        <br>FROM sys.databases        <br>WHERE        <br>&nbsp;&nbsp;&nbsp; database_id IN        <br>&nbsp;&nbsp;&nbsp; (        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SELECT        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; dbid        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; FROM sys.sysprocesses        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; WHERE        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; loginame = 'Adam'        <br>&nbsp;&nbsp;&nbsp; )</font></p>
 </blockquote>
  
<p>If I were to query the various DMVs to get this text without formatting it as XML, the results would be rather ugly once I copied and pasted it out of the SSMS grid:</p>
  
  ![F10_04_bad_paste](img/F10_04_bad_paste.jpg)

<p>XML in SQL Server Management Studio has the nice property of maintaining the formatting of text nodes. Clicking on the hyperlinked XML in the [sql_text] column, rather than attempting to copy and paste, has a much nicer result:</p>
  
  ![F10_04_xml](img/F10_04_xml.jpg)

<p><b>One other consideration is what happens when dealing with nested modules</b>. Consider:</p>
  
<blockquote>   
<p><font face="Courier New">USE tempdb       <br>GO</font></p>
    
<p><font face="Courier New">--Create two "inner" procs</font></p>
    
<p><font face="Courier New">CREATE PROC inner_test_proc1       <br>AS        <br>BEGIN        <br>&nbsp;&nbsp;&nbsp; WAITFOR DELAY '10:10:10'        <br>END        <br>GO</font></p>
    
<p><font face="Courier New">CREATE PROC inner_test_proc2       <br>AS        <br>BEGIN        <br>&nbsp;&nbsp;&nbsp; WAITFOR DELAY '20:20:20'        <br>END        <br>GO</font></p>
    
<p><font face="Courier New">--Create one "outer" proc to call the others</font></p>
    
<p><font face="Courier New">CREATE PROC outer_test_proc       <br>&nbsp;&nbsp;&nbsp; @x INT        <br>AS        <br>BEGIN        <br>&nbsp;&nbsp;&nbsp; IF @x = 1        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; EXEC inner_test_proc1        <br>&nbsp;&nbsp;&nbsp; ELSE        <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; EXEC inner_test_proc2        <br>END        <br>GO</font></p>
    
<p><font face="Courier New">--Call the “outer” test proc       <br></font><font face="Courier New">EXEC outer_test_proc 2       <br>GO</font></p>
 </blockquote>
  
<p>If you run this batch, then run Who is Active, you’ll see output like the following:</p>
  
  ![F10_05_outer1](img/F10_05_outer1.jpg)

<p>The [sql_text] column shows the WAITFOR command currently running. <b>Its value is always scoped to the innermost scope—the currently running batch or module</b>. But what if we want to get a bit more information about what call the application made to get us to this point? To get more information, use the @get_outer_command option:</p>
  
<blockquote>   
<p><font face="Courier New">EXEC sp_WhoIsActive       <br>&nbsp;&nbsp;&nbsp; @get_outer_command = 1</font></p>
 </blockquote>
  
  ![F10_05_outer2](img/F10_05_outer2.jpg)

<p>This option introduces a new column called [sql_command], which reflects the outer batch or stored procedure call that was issued by the application or user. Like the [sql_text] column, it’s clickable XML, so that you can retrieve the original format of the batch. </p>
  
<p><b>Two other considerations for the [sql_text] column</b>:</p>
  
<ul>   
<li>The SQL text in the plan cache will sometimes be locked. Who is Active will wait up to 5ms and then time out, setting the value of the [sql_text] column to “&lt;timeout_exceeded /&gt;”</li>
    
<li>Certain system features, such as automatic statistics updates, may cause surprising results to be returned in the [sql_text] column. For example, rebuilding an index will result in text along the lines of “insert [dbo].[yourTable] select * from [dbo].[yourTable].” If you see something like this, do not be alarmed! Take a deep breath and use the @get_outer_command option to find out what’s really going on.</li>
 </ul>
<hr />